# Initial LCL raw data reading and cleaning 

##### slimmed down - just read in London data and export again in condensed form

ideas from:
https://maxhalford.github.io/blog/streaming-groupbys-in-pandas-for-big-datasets/       
https://stackoverflow.com/questions/42228770/load-pandas-dataframe-with-chunksize-determined-by-column-variable       
https://www.kaggle.com/jeanmidev/smart-meters-in-london/version/7/kernels

In [21]:
# import  module containing all functions written for MSc project
import mscprojectcoding.FM_MScProject_LoadForecasting_Module as FMmodule


In [22]:
# force reimport of module after edits/updates to functions in PyCharm 
import importlib
importlib.reload(FMmodule)

<module 'mscprojectcoding.FM_MScProject_LoadForecasting_Module' from 'c:\\users\\fmurphy\\onedrive - loughborough university\\fm-project\\mscprojectcoding\\src\\mscprojectcoding\\FM_MScProject_LoadForecasting_Module.py'>

In [ ]:
#import functions 
from mscprojectcoding.FM_MScProject_LoadForecasting_Module import  get_limits                 # get x,y limits for plot axis
from mscprojectcoding.FM_MScProject_LoadForecasting_Module import  get_uniques                # get unique values from dataframe columns or pd.series?
from mscprojectcoding.FM_MScProject_LoadForecasting_Module import  show_mem_usage             # shows total memory usage
from mscprojectcoding.FM_MScProject_LoadForecasting_Module import  get_data_range
from mscprojectcoding.FM_MScProject_LoadForecasting_Module import  get_NaN_stats              # get stats for NaNs in each column


In [1]:
import pandas as pd
import time             
import re
import sys
import pickle

filename = "Power-Networks-LCL-June2015 Sample.csv"
filenameALL = "..\Data\London Smartmeter Datset\Power-Networks-LCL-June2015v2.csv" # 11GB


In [3]:
# alternatively , get list of csvs 
# decided not to use this approach as needs further handling of orphaned rows etc
#import glob

# get data file names
#path =r'C:\Users\fmurphy\OneDrive - Loughborough University\FM-Project\Data\London Smartmeter Datset\Power-Networks-LCL-June2015.csv_Pieces'
#filenames = glob.glob(path + "/*_?.csv")
#filenames

In [2]:
# Set max lines etc to display, print etc
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100) # sets limit to print o/p; 
#pd.set_option('display.max_rows', None) # sets no limit to print o/p; only use if reading some of rows

In [ ]:
# Pre processing and checking of data 
# Read 11 GB csv file and process in chunk - reduce data volume

start_time = time.time()

column_names=['meterID', 'tarrif','date_time','kWh30','Acorn','Acorn_Grp']
subset_column_names = ["meterID", "tarrif","Acorn","Acorn_Grp"]   # userID object data (duplicate for each meterID)

#use category dtype for columns with many redundant entries (all subset_column_names)     # x3 reduction in memory usage
dtypes = {'meterID':'category','tarrif':'category','Acorn':'category','Acorn_Grp':'category'}
#dtypes = {'meterID':'str','tarrif':'str','Acorn':'str','Acorn_Grp':'str'}  # just for comparison - memory usage
date_col = ['date_time']

# first count total rows in input file- need this to determine number of chunks for determining orphaned rows later
CHUNKsize = 1000000   
count = 0

#get size of dataset 
for data in pd.read_csv(filenameALL,header=0, iterator=True, names=column_names, chunksize=CHUNKsize,dtype=dtypes):
    # specifying the chunksize argument makes the method call return a TextFileReader object (iterable) rather than DataFrame
    count += 1                          # counting the number of chunks
    lastlen = len(data)                 # finding the length of last chunk   
    
datalength = (count*CHUNKsize + lastlen - CHUNKsize) # length of total file
print("number of chunks: ",count,"number of rows:",datalength, "length of last chunk",lastlen)
last_count=count
print("chunk count: ",last_count)
del data                # release memory used by 'data'

#create empty df for o/p
orphans = pd.DataFrame()                # to store left over rows at end of each chunk
df_unique_users = pd.DataFrame()        # for list of unique users with corresponding tarrif, Accorn grp etc
df_selected_data = pd.DataFrame()       # output data

#define date range to keep in output e.g. all of 2013 - data starts/ends at different times but for most meters, spans all 2013
start = '2013-01-01 00:00:00'
end = '2014-01-01 00:00:00'

iterator = pd.read_csv(filenameALL,header=0,iterator=True,names=column_names,chunksize=CHUNKsize,dtype=dtypes,parse_dates=date_col)
count = 0

for df in iterator:                                                     
    count +=1     # increment at start for if test below
    
    #remove rows where load data NULL - ignore for now (none?)
    #df = df[df['kWh30'] != 'Null']    

    #get list of unique userIDs (meterID + ancillary data eg Acorn group) and append to
    uniques = get_uniques(df,subset_column_names)
    df_unique_users = pd.concat([df_unique_users,uniques])
    
    # drop ancillary data from df
    df.drop(columns=['tarrif','Acorn','Acorn_Grp'], inplace = True) 
    
    # convert load data (pandas object) to numeric
    #https://stackoverflow.com/questions/40095712/when-to-applypd-to-numeric-and-when-to-astypenp-float64-in-python    
    df['kWh30'] = pd.to_numeric(df['kWh30'],errors='coerce')               
    
    #not sure if this still needed given parse_dates in pd.read_csv ?
    df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')    
    
    #set date_time as index - testing
    #df = df.set_index('date_time')
    
    # strip prefix 'MAC' from meterID and convert to numeric
    df['meterID'] = pd.to_numeric(df['meterID'].replace('MAC','', regex=True), errors='coerce')
    #print(sys.getsizeof(df))   # check memory usage after setting dtype categorical
      
    # concat orphans from previous iteration  (none for first iteration)
    df = pd.concat([orphans,df])
    
    # find which rows are orphans
    last_val = df['meterID'].iloc[-1]          # get meterID of last row of chunk i.e. the one to be orphaned 
    is_orphan = df['meterID'] == last_val      # true if matches orphan meterID ie for all orphan rows in chunk
    
    # put the new orphans aside if not the last chunk,
    # for last chunk, want to keep all remaining lines (assume that will eventually run this on the complete 11GB file) 
    if (count < last_count):
        df, orphans = df[~is_orphan], df[is_orphan]        

    #only keep data in start:end date range from main df
    df_selected_data = pd.concat([df_selected_data,df[ (df['date_time'] >= start) & (df['date_time'] < end) ]])  
    
    
    #show_mem_usage()  # to get idea of how much increments with each chunk (SLOW!)
    
    
    print("Chunk count is:",count)
    # break out of reading chunks after 50 read - in case run out of memory..
    #if (count > 20):
    #    break

del df
del is_orphan

#get range of datetime for each userID - run first time to get idea of time spans
#df_datetime_bounds.rename(columns={'': 'min_datetime', '': 'max_datetime'}, inplace=True)  # mnot working..
#df_datetime_bounds = get_data_range(df,'meterID','date_time'
#df_datetime_bounds.to_csv('London_userID_DateTimeRange.csv')

#run unique filter again on usereID dataframe - may have some duplicates from different chunks; 
df_unique_users = get_uniques(df_unique_users,subset_column_names)
df_unique_users.to_csv('London_userID_Summary.csv')


#check output dataframe
#print(df_selected_data.shape)   
#print(df_selected_data.head(10))
#print(df_selected_data.tail(10))
#print(df_selected_data.info)
#print(df_selected_data.describe)
#print(df_selected_data.dtypes)

#find rows with null load data (if any)
null_data = df_selected_data[df_selected_data['kWh30'] == 'Null']
null_data.to_csv('null_data.csv')

#check memory use at end 
show_mem_usage() 

# EXPORT SELECTED DATA TO PICKLE FILE - use inbuilt pandas pickle method
# https://stackoverflow.com/questions/28754658/whats-the-fastest-way-to-pickle-a-pandas-dataframe
df_selected_data.to_pickle('./cleaned_London_data.pkl')

# calculate execution time and print
print("--- %s seconds ---" % (time.time() - start_time)) 


In [7]:
#load pickle file from above
infile = open('../MScProjectCoding/project_data/intermediateDataLCL/cleaned_London_data.pkl','rb')
#infile = open('./cleaned_London_data.pkl','rb')
df_selected_data = pickle.load(infile)
infile.close()
df_selected_data

,meterID,date_time,kWh30
3840,2,2013-01-01 00:00:00,0.219
3841,2,2013-01-01 00:30:00,0.241
3842,2,2013-01-01 01:00:00,0.191
3843,2,2013-01-01 01:30:00,0.235
3844,2,2013-01-01 02:00:00,0.182
...,...,...,...
167929637,5564,2013-12-31 21:30:00,0.089
167929638,5564,2013-12-31 22:00:00,0.092
167929639,5564,2013-12-31 22:30:00,0.059
167929640,5564,2013-12-31 23:00:00,0.086


In [10]:
# re-read unique user csv
df_unique_users = pd.read_csv('./London_userID_Summary.csv')

FileNotFoundError: [Errno 2] File ./London_userID_Summary.csv does not exist: './London_userID_Summary.csv'

In [8]:
#NaN_stats
 
# first , get nrows Total
print("Total Nrows in df_selected_data: ",df_selected_data.shape[0])

#print(get_NaN_stats(df_selected_data)) 
print("NaN stats: ")
df_selected_data.isnull().sum(axis=0)
df_selected_data[df_selected_data.isnull().any(axis=1)]


Total Nrows in df_selected_data:  93087837
NaN stats: 


,meterID,date_time,kWh30
37956017,1478,2013-03-08 18:19:44,NaN
57802006,2328,2013-07-05 13:15:05,NaN
64917844,2629,2013-04-16 18:15:40,NaN
69030451,2796,2013-02-13 18:14:54,NaN
95010182,4069,2013-07-05 13:15:05,NaN
99511582,4248,2013-02-06 18:22:38,NaN
120804779,5037,2013-01-04 18:20:32,NaN
143691393,1477,2013-10-29 09:33:01,NaN
144709892,1653,2013-10-29 09:33:01,NaN
153932659,3155,2013-07-05 12:54:31,NaN


In [9]:
df_selected_data

,meterID,date_time,kWh30
3840,2,2013-01-01 00:00:00,0.219
3841,2,2013-01-01 00:30:00,0.241
3842,2,2013-01-01 01:00:00,0.191
3843,2,2013-01-01 01:30:00,0.235
3844,2,2013-01-01 02:00:00,0.182
...,...,...,...
167929637,5564,2013-12-31 21:30:00,0.089
167929638,5564,2013-12-31 22:00:00,0.092
167929639,5564,2013-12-31 22:30:00,0.059
167929640,5564,2013-12-31 23:00:00,0.086


In [4]:
show_mem_usage() 

0.005MB


In [11]:
# drop duplicates from above
# there are duplicate readings at mid-night some days (same time stamp and load data) - will be removed above if same load reading

df_selected_data_noDups = df_selected_data.drop_duplicates()
#df_selected_data_noDups.to_csv('data_sample_noDups.csv')
df_selected_data_noDups.shape

# NOTE not fully sorted - meterIDs are grouped together but groups are in two lots (tarrifs?)



(93024229, 3)

In [12]:
# get nrows per meterID : expect 17520 ie (2*24*365)
df_row_count = pd.DataFrame(df_selected_data_noDups.groupby('meterID')['date_time'].count())
df_row_count.columns = ['value_count']
print("nrows per meterID: ",df_row_count)

#select meterID with no missing times
meterID_keep = df_row_count.loc[df_row_count['value_count'] == 17520]  #.index same as meterID
meterID_keep.to_csv('meterID_keep.csv')
meterID_keep                                # this is sorted by meterID


nrows per meterID:           value_count
meterID             
2              17520
3              17518
4              17519
5              17519
6              17520
...              ...
5561           17515
5562           17519
5564           17511
5566           17519
5567           17516

[5528 rows x 1 columns]


,value_count
meterID,
2,17520
6,17520
10,17520
18,17520
19,17520
...,...
5538,17520
5539,17520
5540,17520


In [14]:
df_row_count.sort_values(by=['value_count'])

,value_count
meterID,
2976,385
5092,385
3353,433
1338,433
4612,433
...,...
3160,17520
3158,17520
3154,17520


In [15]:
# find if any  gaps in date_time 

# most gaps are 1 hr (one missing reading in input file), a few are longer eg 6 hrs
 
# find for each meterID, if gaps in date_time 
#  preserves original index?   YES
df_time_gaps = df_selected_data_noDups.loc[(df_selected_data_noDups['meterID'] == df_selected_data_noDups['meterID'].shift(-1))&(df_selected_data_noDups['date_time'].diff() > '00:30:00')]
df_time_gaps.to_csv('time_gaps.csv')

# where time gap, extract preceeding row (for checking)
#df_selected_data_Gaps = df_selected_data_noDups[df_selected_data_noDups.index.isin((df_time_gaps.index)-1)]

print("Nrows (with time gaps):", df_time_gaps.shape[0])

# count for each meterID, number of rows with gaps 
df_time_gaps.groupby('meterID')['date_time'].count()
#print(df_time_gaps.shape)

# count total number of meterID with gaps 
df_time_gaps_uniqueID = df_time_gaps.drop_duplicates(subset='meterID')
print("total number of meterID with gaps  : ",df_time_gaps_uniqueID.shape[0])     

print("time gaps: :",df_time_gaps)

Nrows (with time gaps): 11924
total number of meterID with gaps  :  4202
time gaps: :            meterID           date_time  kWh30
43388            3 2013-03-26 22:00:00  0.168
45420            3 2013-05-08 06:00:00  0.584
76906            4 2013-05-04 06:30:00  0.000
134731           7 2013-02-16 13:00:00  0.094
149765           7 2013-12-26 13:30:00  0.085
...            ...                 ...    ...
167894334     5557 2013-07-24 08:00:00  0.079
167894355     5557 2013-07-24 19:00:00  0.104
167901128     5557 2013-12-12 19:30:00  0.071
167913337     5564 2013-01-26 09:00:00  0.110
167915643     5564 2013-03-15 13:30:00  0.096

[11924 rows x 3 columns]


In [16]:
# now subset data to only keep where full set of data for 2013 (no time gaps) i.e. for meterID in array meterID_keep

df_selected_data_Complete = df_selected_data_noDups.loc[df_selected_data_noDups['meterID'].isin(meterID_keep.index)]
print(" Subset Data size :  ",df_selected_data_Complete.shape)
df_selected_data_Complete

 Subset Data size :   (19832640, 3)


,meterID,date_time,kWh30
3840,2,2013-01-01 00:00:00,0.219
3841,2,2013-01-01 00:30:00,0.241
3842,2,2013-01-01 01:00:00,0.191
3843,2,2013-01-01 01:30:00,0.235
3844,2,2013-01-01 02:00:00,0.182
...,...,...,...
167811964,5549,2013-12-31 21:30:00,0.259
167811965,5549,2013-12-31 22:00:00,0.227
167811966,5549,2013-12-31 22:30:00,0.275
167811967,5549,2013-12-31 23:00:00,0.212


In [17]:
# get unique meterIDs from above and check consistent with meterID_keep

uniques = df_selected_data_Complete.drop_duplicates(subset='meterID')
uniques.to_csv('uniques.csv')

# compared csvs - ok once sorted

In [18]:
# check if any 0 time gaps left (e.g. if had same time stamp but different load values)
df_selected_data_noDups.loc[df_selected_data_noDups['date_time'].diff() == '00:00:00']

,meterID,date_time,kWh30


In [19]:
#repeat check for time gaps  - should have none 

df_time_gaps_2 = df_selected_data_Complete.loc[(df_selected_data_Complete['meterID'] == df_selected_data_Complete['meterID'].shift(-1))&(df_selected_data_Complete['date_time'].diff() > '00:30:00')]

print("Nrows (with time gaps):", df_time_gaps_2.shape[0])

# count for each meterID, number of rows with gaps 
df_time_gaps_2.groupby('meterID')['date_time'].count()
#print(df_time_gaps_2.shape)

# count total number of meterID with gaps 
df_time_gaps_uniqueID = df_time_gaps_2.drop_duplicates(subset='meterID')
print("total number of meterID with gaps  : ",df_time_gaps_uniqueID.shape[0])     

print("time gaps: :",df_time_gaps_2)

Nrows (with time gaps): 0
total number of meterID with gaps  :  0
time gaps: : Empty DataFrame
Columns: [meterID, date_time, kWh30]
Index: []


In [ ]:

#df_joined_indices=pd.merge(df_time_gaps,df_selected_data_Gaps,on=df_time_gaps.index, how='outer', validate="one_to_one")
#df_joined=pd.concat([df_time_gaps, df_selected_data_Gaps], axis=1)
#df_joined.to_csv('df_joined.csv')


In [20]:
# verify complete dataset for each meterID

# get nrows per meterID : expect 17520 ie (2*24*365) 

df_row_count = pd.DataFrame(df_selected_data_Complete.groupby('meterID')['date_time'].count())
df_row_count.columns = ['value_count']
print("nrows per meterID: ",df_row_count)

nrows per meterID:           value_count
meterID             
2              17520
6              17520
10             17520
18             17520
19             17520
...              ...
5538           17520
5539           17520
5540           17520
5549           17520
5552           17520

[1132 rows x 1 columns]


In [ ]:
# SKIP easier to do with pd.pivot

# now unstack data (group by meterID and transpose so have : meterID, data for each of 17520 timestamp columns)
#df_selected_data_rearranged = df_selected_data_Complete.groupby('meterID')['kWh30'].apply(lambda df: df.reset_index(drop=True)).unstack()
#df_selected_data_rearranged

In [49]:
# unstack/rearrange data
#df_selected_data_rearranged = df_selected_data_Complete.pivot(index='meterID',columns='date_time') 

df_selected_data_rearranged.columns = [col[1] for col in df_selected_data_rearranged.columns]
df_selected_data_rearranged

,2013-01-01 00:00:00,2013-01-01 00:30:00,2013-01-01 01:00:00,2013-01-01 01:30:00,2013-01-01 02:00:00,2013-01-01 02:30:00,2013-01-01 03:00:00,2013-01-01 03:30:00,2013-01-01 04:00:00,2013-01-01 04:30:00,2013-01-01 05:00:00,2013-01-01 05:30:00,2013-01-01 06:00:00,2013-01-01 06:30:00,2013-01-01 07:00:00,2013-01-01 07:30:00,2013-01-01 08:00:00,2013-01-01 08:30:00,2013-01-01 09:00:00,2013-01-01 09:30:00,2013-01-01 10:00:00,2013-01-01 10:30:00,2013-01-01 11:00:00,2013-01-01 11:30:00,2013-01-01 12:00:00,...,2013-12-31 11:30:00,2013-12-31 12:00:00,2013-12-31 12:30:00,2013-12-31 13:00:00,2013-12-31 13:30:00,2013-12-31 14:00:00,2013-12-31 14:30:00,2013-12-31 15:00:00,2013-12-31 15:30:00,2013-12-31 16:00:00,2013-12-31 16:30:00,2013-12-31 17:00:00,2013-12-31 17:30:00,2013-12-31 18:00:00,2013-12-31 18:30:00,2013-12-31 19:00:00,2013-12-31 19:30:00,2013-12-31 20:00:00,2013-12-31 20:30:00,2013-12-31 21:00:00,2013-12-31 21:30:00,2013-12-31 22:00:00,2013-12-31 22:30:00,2013-12-31 23:00:00,2013-12-31 23:30:00
meterID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.219,0.241,0.191,0.235,0.182,0.229,0.194,0.201,0.122,0.099,0.123,0.083,0.130,0.076,0.126,0.091,0.110,0.106,0.094,0.122,0.076,0.124,0.122,0.120,0.794,...,0.332,0.379,0.243,0.127,0.103,0.106,0.514,0.581,0.453,0.394,0.447,0.377,0.370,0.195,0.282,0.252,0.307,0.833,0.400,0.332,0.451,0.456,0.402,0.491,0.452
6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.023,0.045,0.045,0.045,0.044,0.025,0.000,0.000,0.000,0.000,0.000,...,0.049,0.058,0.032,0.069,0.039,0.044,0.064,0.141,0.076,0.063,0.058,0.097,0.058,0.070,0.209,0.100,0.127,0.116,0.102,0.142,0.067,0.076,0.078,0.040,0.073
10,0.509,0.453,0.500,0.621,0.197,0.176,0.210,0.211,0.151,0.152,0.236,0.214,0.184,0.205,0.245,0.207,0.208,0.495,0.267,0.245,0.409,0.345,1.290,0.827,0.284,...,0.762,0.483,0.595,0.754,0.799,0.793,1.777,1.524,2.333,1.703,1.656,1.494,1.387,2.470,1.105,1.016,1.558,1.391,1.198,0.932,0.715,0.697,0.779,0.851,0.850
18,0.187,0.191,0.131,0.146,0.115,0.142,0.133,0.124,0.161,0.253,0.481,0.270,0.288,0.271,0.281,0.251,0.304,0.206,0.193,0.145,0.130,0.122,0.143,0.122,0.129,...,0.110,0.109,0.238,0.228,0.230,0.236,0.321,0.157,0.135,0.134,0.334,0.443,0.661,0.554,0.555,0.565,0.292,0.127,0.125,0.128,0.145,0.135,0.132,0.139,0.134
19,0.195,0.087,0.116,0.069,0.109,0.068,0.109,0.028,0.052,0.029,0.061,0.091,0.085,0.095,0.104,0.122,0.050,0.490,0.252,0.182,0.178,0.188,0.182,0.170,0.176,...,0.230,0.197,0.165,0.539,0.230,0.239,0.206,0.231,0.236,0.290,0.230,0.285,0.426,0.244,0.219,0.206,0.197,0.218,0.219,0.218,0.190,0.221,0.211,0.194,0.220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5538,0.020,0.018,0.018,0.042,0.040,0.040,0.013,0.012,0.010,0.011,0.010,0.011,0.010,0.010,0.010,0.010,0.011,0.010,0.029,0.041,0.040,0.035,0.010,0.011,0.010,...,0.197,0.187,0.199,0.198,0.200,0.199,0.046,0.389,0.169,0.182,0.439,0.101,0.099,0.120,0.160,0.126,0.068,0.031,0.219,0.202,0.276,0.402,0.217,0.395,0.173
5539,0.499,0.440,0.300,0.319,0.215,0.187,0.138,0.136,0.174,0.183,0.122,0.180,0.214,0.176,0.212,0.224,0.318,0.216,0.342,0.220,0.282,0.465,0.248,0.251,0.263,...,0.644,0.266,0.493,0.498,0.433,0.339,0.379,0.458,0.358,0.278,0.326,0.345,0.415,0.344,0.295,0.470,0.585,0.640,1.157,1.574,1.153,0.451,0.714,0.508,0.381
5540,0.059,0.032,0.069,0.062,0.030,0.063,0.067,0.030,0.056,0.066,0.031,0.057,0.066,0.030,0.057,0.062,0.030,0.061,0.100,0.077,0.074,0.071,0.054,0.031,0.061,...,0.061,0.040,0.108,0.064,0.266,0.103,0.049,0.104,0.438,0.359,0.084,0.050,0.080,0.071,0.037,0.060,0.028,0.061,0.037,0.045,0.053,0.028,0.069,0.028,0.051


In [52]:
# check sorting
df_selected_data_rearranged.index

Int64Index([   2,    6,   10,   18,   19,   27,   40,   48,   49,   57,
            ...
            5525, 5526, 5527, 5530, 5535, 5538, 5539, 5540, 5549, 5552],
           dtype='int64', name='meterID', length=1132)

In [53]:
# now add back in other info on user eg tarrif and Acorn group etc
# first - read in userID info
df_unique_users = pd.read_csv('./London_userID_Summary.csv')
df_unique_users.drop(columns=['date_time','kWh30'], inplace = True) 
df_unique_users['meterID'] = pd.to_numeric(df_unique_users['meterID'].replace('MAC','', regex=True), errors='coerce')
df_unique_users.drop(df_unique_users.columns[df_unique_users.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

# keep only meterID_keep  ie same meterID as in df_selected_data_rearranged
df_unique_users = df_unique_users.loc[df_unique_users['meterID'].isin(meterID_keep.index)]
df_unique_users = df_unique_users.set_index('meterID')

# sort by meterID (was in 2 groups)
df_unique_users = df_unique_users.sort_values(by=['meterID'])  
df_unique_users





,tarrif,Acorn,Acorn_Grp
meterID,,,
2,Std,ACORN-A,Affluent
6,Std,ACORN-Q,Adversity
10,Std,ACORN-H,Comfortable
18,Std,ACORN-E,Affluent
19,Std,ACORN-K,Adversity
...,...,...,...
5538,ToU,ACORN-F,Comfortable
5539,Std,ACORN-E,Affluent
5540,Std,ACORN-N,Adversity


In [54]:
 # is sorted I think but sort anyway..
df_selected_data_rearranged = df_selected_data_rearranged.sort_values(by=['meterID'])   
df_selected_data_rearranged
df_selected_data_rearranged.columns


DatetimeIndex(['2013-01-01 00:00:00', '2013-01-01 00:30:00',
               '2013-01-01 01:00:00', '2013-01-01 01:30:00',
               '2013-01-01 02:00:00', '2013-01-01 02:30:00',
               '2013-01-01 03:00:00', '2013-01-01 03:30:00',
               '2013-01-01 04:00:00', '2013-01-01 04:30:00',
               ...
               '2013-12-31 19:00:00', '2013-12-31 19:30:00',
               '2013-12-31 20:00:00', '2013-12-31 20:30:00',
               '2013-12-31 21:00:00', '2013-12-31 21:30:00',
               '2013-12-31 22:00:00', '2013-12-31 22:30:00',
               '2013-12-31 23:00:00', '2013-12-31 23:30:00'],
              dtype='datetime64[ns]', length=17520, freq=None)

In [55]:
#pd.concat([df_unique_users,df_selected_data_rearranged],axis = 1)
#df_unique_users.join(df_selected_data_rearrange,how='inner')
df_merged = df_unique_users.merge(df_selected_data_rearranged, left_index=True, right_index=True, how='inner')
df_merged

,tarrif,Acorn,Acorn_Grp,2013-01-01 00:00:00,2013-01-01 00:30:00,2013-01-01 01:00:00,2013-01-01 01:30:00,2013-01-01 02:00:00,2013-01-01 02:30:00,2013-01-01 03:00:00,2013-01-01 03:30:00,2013-01-01 04:00:00,2013-01-01 04:30:00,2013-01-01 05:00:00,2013-01-01 05:30:00,2013-01-01 06:00:00,2013-01-01 06:30:00,2013-01-01 07:00:00,2013-01-01 07:30:00,2013-01-01 08:00:00,2013-01-01 08:30:00,2013-01-01 09:00:00,2013-01-01 09:30:00,2013-01-01 10:00:00,2013-01-01 10:30:00,...,2013-12-31 11:30:00,2013-12-31 12:00:00,2013-12-31 12:30:00,2013-12-31 13:00:00,2013-12-31 13:30:00,2013-12-31 14:00:00,2013-12-31 14:30:00,2013-12-31 15:00:00,2013-12-31 15:30:00,2013-12-31 16:00:00,2013-12-31 16:30:00,2013-12-31 17:00:00,2013-12-31 17:30:00,2013-12-31 18:00:00,2013-12-31 18:30:00,2013-12-31 19:00:00,2013-12-31 19:30:00,2013-12-31 20:00:00,2013-12-31 20:30:00,2013-12-31 21:00:00,2013-12-31 21:30:00,2013-12-31 22:00:00,2013-12-31 22:30:00,2013-12-31 23:00:00,2013-12-31 23:30:00
meterID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Std,ACORN-A,Affluent,0.219,0.241,0.191,0.235,0.182,0.229,0.194,0.201,0.122,0.099,0.123,0.083,0.130,0.076,0.126,0.091,0.110,0.106,0.094,0.122,0.076,0.124,...,0.332,0.379,0.243,0.127,0.103,0.106,0.514,0.581,0.453,0.394,0.447,0.377,0.370,0.195,0.282,0.252,0.307,0.833,0.400,0.332,0.451,0.456,0.402,0.491,0.452
6,Std,ACORN-Q,Adversity,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.023,0.045,0.045,0.045,0.044,0.025,0.000,0.000,...,0.049,0.058,0.032,0.069,0.039,0.044,0.064,0.141,0.076,0.063,0.058,0.097,0.058,0.070,0.209,0.100,0.127,0.116,0.102,0.142,0.067,0.076,0.078,0.040,0.073
10,Std,ACORN-H,Comfortable,0.509,0.453,0.500,0.621,0.197,0.176,0.210,0.211,0.151,0.152,0.236,0.214,0.184,0.205,0.245,0.207,0.208,0.495,0.267,0.245,0.409,0.345,...,0.762,0.483,0.595,0.754,0.799,0.793,1.777,1.524,2.333,1.703,1.656,1.494,1.387,2.470,1.105,1.016,1.558,1.391,1.198,0.932,0.715,0.697,0.779,0.851,0.850
18,Std,ACORN-E,Affluent,0.187,0.191,0.131,0.146,0.115,0.142,0.133,0.124,0.161,0.253,0.481,0.270,0.288,0.271,0.281,0.251,0.304,0.206,0.193,0.145,0.130,0.122,...,0.110,0.109,0.238,0.228,0.230,0.236,0.321,0.157,0.135,0.134,0.334,0.443,0.661,0.554,0.555,0.565,0.292,0.127,0.125,0.128,0.145,0.135,0.132,0.139,0.134
19,Std,ACORN-K,Adversity,0.195,0.087,0.116,0.069,0.109,0.068,0.109,0.028,0.052,0.029,0.061,0.091,0.085,0.095,0.104,0.122,0.050,0.490,0.252,0.182,0.178,0.188,...,0.230,0.197,0.165,0.539,0.230,0.239,0.206,0.231,0.236,0.290,0.230,0.285,0.426,0.244,0.219,0.206,0.197,0.218,0.219,0.218,0.190,0.221,0.211,0.194,0.220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5538,ToU,ACORN-F,Comfortable,0.020,0.018,0.018,0.042,0.040,0.040,0.013,0.012,0.010,0.011,0.010,0.011,0.010,0.010,0.010,0.010,0.011,0.010,0.029,0.041,0.040,0.035,...,0.197,0.187,0.199,0.198,0.200,0.199,0.046,0.389,0.169,0.182,0.439,0.101,0.099,0.120,0.160,0.126,0.068,0.031,0.219,0.202,0.276,0.402,0.217,0.395,0.173
5539,Std,ACORN-E,Affluent,0.499,0.440,0.300,0.319,0.215,0.187,0.138,0.136,0.174,0.183,0.122,0.180,0.214,0.176,0.212,0.224,0.318,0.216,0.342,0.220,0.282,0.465,...,0.644,0.266,0.493,0.498,0.433,0.339,0.379,0.458,0.358,0.278,0.326,0.345,0.415,0.344,0.295,0.470,0.585,0.640,1.157,1.574,1.153,0.451,0.714,0.508,0.381
5540,Std,ACORN-N,Adversity,0.059,0.032,0.069,0.062,0.030,0.063,0.067,0.030,0.056,0.066,0.031,0.057,0.066,0.030,0.057,0.062,0.030,0.061,0.100,0.077,0.074,0.071,...,0.061,0.040,0.108,0.064,0.266,0.103,0.049,0.104,0.438,0.359,0.084,0.050,0.080,0.071,0.037,0.060,0.028,0.061,0.037,0.045,0.053,0.028,0.069,0.028,0.051


In [57]:
df_merged['user_id']=df_merged.index

cols = list(df_merged.columns)
cols = [cols[-1]] + cols[:-1]
df_merged = df_merged[cols]

df_merged

,user_id,tarrif,Acorn,Acorn_Grp,2013-01-01 00:00:00,2013-01-01 00:30:00,2013-01-01 01:00:00,2013-01-01 01:30:00,2013-01-01 02:00:00,2013-01-01 02:30:00,2013-01-01 03:00:00,2013-01-01 03:30:00,2013-01-01 04:00:00,2013-01-01 04:30:00,2013-01-01 05:00:00,2013-01-01 05:30:00,2013-01-01 06:00:00,2013-01-01 06:30:00,2013-01-01 07:00:00,2013-01-01 07:30:00,2013-01-01 08:00:00,2013-01-01 08:30:00,2013-01-01 09:00:00,2013-01-01 09:30:00,2013-01-01 10:00:00,...,2013-12-31 11:30:00,2013-12-31 12:00:00,2013-12-31 12:30:00,2013-12-31 13:00:00,2013-12-31 13:30:00,2013-12-31 14:00:00,2013-12-31 14:30:00,2013-12-31 15:00:00,2013-12-31 15:30:00,2013-12-31 16:00:00,2013-12-31 16:30:00,2013-12-31 17:00:00,2013-12-31 17:30:00,2013-12-31 18:00:00,2013-12-31 18:30:00,2013-12-31 19:00:00,2013-12-31 19:30:00,2013-12-31 20:00:00,2013-12-31 20:30:00,2013-12-31 21:00:00,2013-12-31 21:30:00,2013-12-31 22:00:00,2013-12-31 22:30:00,2013-12-31 23:00:00,2013-12-31 23:30:00
meterID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2,Std,ACORN-A,Affluent,0.219,0.241,0.191,0.235,0.182,0.229,0.194,0.201,0.122,0.099,0.123,0.083,0.130,0.076,0.126,0.091,0.110,0.106,0.094,0.122,0.076,...,0.332,0.379,0.243,0.127,0.103,0.106,0.514,0.581,0.453,0.394,0.447,0.377,0.370,0.195,0.282,0.252,0.307,0.833,0.400,0.332,0.451,0.456,0.402,0.491,0.452
6,6,Std,ACORN-Q,Adversity,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.023,0.045,0.045,0.045,0.044,0.025,0.000,...,0.049,0.058,0.032,0.069,0.039,0.044,0.064,0.141,0.076,0.063,0.058,0.097,0.058,0.070,0.209,0.100,0.127,0.116,0.102,0.142,0.067,0.076,0.078,0.040,0.073
10,10,Std,ACORN-H,Comfortable,0.509,0.453,0.500,0.621,0.197,0.176,0.210,0.211,0.151,0.152,0.236,0.214,0.184,0.205,0.245,0.207,0.208,0.495,0.267,0.245,0.409,...,0.762,0.483,0.595,0.754,0.799,0.793,1.777,1.524,2.333,1.703,1.656,1.494,1.387,2.470,1.105,1.016,1.558,1.391,1.198,0.932,0.715,0.697,0.779,0.851,0.850
18,18,Std,ACORN-E,Affluent,0.187,0.191,0.131,0.146,0.115,0.142,0.133,0.124,0.161,0.253,0.481,0.270,0.288,0.271,0.281,0.251,0.304,0.206,0.193,0.145,0.130,...,0.110,0.109,0.238,0.228,0.230,0.236,0.321,0.157,0.135,0.134,0.334,0.443,0.661,0.554,0.555,0.565,0.292,0.127,0.125,0.128,0.145,0.135,0.132,0.139,0.134
19,19,Std,ACORN-K,Adversity,0.195,0.087,0.116,0.069,0.109,0.068,0.109,0.028,0.052,0.029,0.061,0.091,0.085,0.095,0.104,0.122,0.050,0.490,0.252,0.182,0.178,...,0.230,0.197,0.165,0.539,0.230,0.239,0.206,0.231,0.236,0.290,0.230,0.285,0.426,0.244,0.219,0.206,0.197,0.218,0.219,0.218,0.190,0.221,0.211,0.194,0.220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5538,5538,ToU,ACORN-F,Comfortable,0.020,0.018,0.018,0.042,0.040,0.040,0.013,0.012,0.010,0.011,0.010,0.011,0.010,0.010,0.010,0.010,0.011,0.010,0.029,0.041,0.040,...,0.197,0.187,0.199,0.198,0.200,0.199,0.046,0.389,0.169,0.182,0.439,0.101,0.099,0.120,0.160,0.126,0.068,0.031,0.219,0.202,0.276,0.402,0.217,0.395,0.173
5539,5539,Std,ACORN-E,Affluent,0.499,0.440,0.300,0.319,0.215,0.187,0.138,0.136,0.174,0.183,0.122,0.180,0.214,0.176,0.212,0.224,0.318,0.216,0.342,0.220,0.282,...,0.644,0.266,0.493,0.498,0.433,0.339,0.379,0.458,0.358,0.278,0.326,0.345,0.415,0.344,0.295,0.470,0.585,0.640,1.157,1.574,1.153,0.451,0.714,0.508,0.381
5540,5540,Std,ACORN-N,Adversity,0.059,0.032,0.069,0.062,0.030,0.063,0.067,0.030,0.056,0.066,0.031,0.057,0.066,0.030,0.057,0.062,0.030,0.061,0.100,0.077,0.074,...,0.061,0.040,0.108,0.064,0.266,0.103,0.049,0.104,0.438,0.359,0.084,0.050,0.080,0.071,0.037,0.060,0.028,0.061,0.037,0.045,0.053,0.028,0.069,0.028,0.051


In [58]:
#export to pickle
df_merged.to_pickle('./cleaned_London_data_subset.pkl')